In [6]:
import tiktoken
import torch
import torch.nn as nn

In [11]:
encoding_model = tiktoken.get_encoding('p50k_base')

In [13]:
vocab_size = encoding_model.n_vocab

In [14]:
embedding_layer = nn.Embedding(vocab_size, 512)

In [21]:
sample_encoding = encoding_model.encode('SELECT * FROM sales')

In [25]:
sample_encoding

[46506, 1635, 16034, 4200]

In [23]:
sample_embedding = embedding_layer(torch.tensor(sample_encoding))

In [24]:
sample_embedding.shape

torch.Size([4, 512])